In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q keras

In [ ]:
pip install pyyaml h5py  # Required to save models in HDF5 format

In [3]:
import os
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense,Flatten
from keras.models import Model, Sequential,load_model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG19
from keras.callbacks import ModelCheckpoint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

In [4]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 horizontal_flip=True,shear_range=.2,rotation_range=.2)
test_datagen=ImageDataGenerator(rescale=1./255)


In [5]:
training_set=train_datagen.flow_from_directory("/content/drive/MyDrive/train",
                                               target_size=(224,224),class_mode="categorical",batch_size=128,shuffle=True)
test_set=test_datagen.flow_from_directory("/content/drive/MyDrive/test",
                                          target_size=(224,224),batch_size=128,class_mode="categorical",shuffle=True)

Found 16926 images belonging to 4 classes.
Found 1000 images belonging to 4 classes.


In [6]:
vgg19=VGG19(include_top=False,weights="imagenet",input_shape=[224,224,3])


80134624/80134624 [==============================] - 0s 0us/step


In [7]:
for layer in vgg19.layers:
  layer.trainable=False

In [8]:
folder=glob("/content/drive/MyDrive/train/*")
folder

['/content/drive/MyDrive/train/CNV',
 '/content/drive/MyDrive/train/DME',
 '/content/drive/MyDrive/train/DRUSEN',
 '/content/drive/MyDrive/train/NORMAL']

In [9]:
x=Flatten()(vgg19.output)
pred_vgg19=Dense(units=len(folder),activation="softmax")(x)
vgg19_model=Model(inputs=vgg19.input,outputs=pred_vgg19)


In [10]:
vgg19_model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [11]:
checkpoint_callback = ModelCheckpoint('model_checkpoint_epoch_{epoch:02d}.h5',
                                       save_weights_only=True)

In [12]:
vgg19_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
#checkpoint_path = "training_1/cp.ckpt"
#checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
#cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 #save_weights_only=True,
                                                 #verbose=1)


In [ ]:
total_epochs = 5  # Change this to the total number of epochs you want to train for
#model.fit(x_train, y_train, epochs=total_epochs, validation_data=(x_test, y_test),
          #callbacks=[checkpoint_callback])

transfer_vgg19=vgg19_model.fit(training_set,validation_data=test_set,epochs=total_epochs,
                                         validation_steps=len(test_set),steps_per_epoch=len(training_set), callbacks=[checkpoint_callback])

Epoch 1/5
133/133 [==============================] - 14254s 107s/step - loss: 0.5837 - accuracy: 0.7587 - val_loss: 8.1719 - val_accuracy: 0.6570
Epoch 2/5
 18/133 [===>..........................] - ETA: 3:04:49 - loss: 0.4372 - accuracy: 0.8228

In [ ]:
checkpoint_files = glob.glob('model_checkpoint_epoch_*.h5')

if not checkpoint_files:
    print("No checkpoint files found. Training from scratch.")
else:
    # Load the latest checkpoint if available
    latest_checkpoint = max(checkpoint_files, key=os.path.getctime)
    vgg19_model.load_weights(latest_checkpoint)

No checkpoint files found. Training from scratch.


In [ ]:
transfer_vgg19.load_weights('model_checkpoint_epoch_03.h5')

transfer_vgg19=vgg19_model.fit(training_set,validation_data=test_set,epochs=total_epochs, initial_epoch=10,
                                         validation_steps=len(test_set),steps_per_epoch=len(training_set), callbacks=[checkpoint_callback])

#model.fit(x_train, y_train, epochs=total_epochs, validation_data=(x_test, y_test),
          #initial_epoch=,  # Start from epoch 5
          #callbacks=[checkpoint_callback])